In [1]:
import logging
from copy import deepcopy

import numpy as np
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from regression import CustomRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# conf = SparkConf()
# spark = SparkContext.getOrCreate(conf=conf)

In [2]:
train = pd.read_pickle('train_df.p')
test = pd.read_pickle('test_df.p')

keys = set(train.keys())
keys.remove('000001.SS')
keys.remove('Target')
ass = VectorAssembler(inputCols=list(keys), outputCol="features")
df = spark.createDataFrame(train)
output = ass.transform(df)
test_df = spark.createDataFrame(test)
test_fe = ass.transform(test_df)

In [9]:
lr = LinearRegression(featuresCol="features", labelCol='Target', maxIter=10000)
model = lr.fit(output)
test_re = model.transform(test_fe)

In [6]:
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()), 
                      output.select('Target').rdd.map(lambda x: list(x)[0]))

In [7]:
# data_df = test_fe.select('features').toPandas()
model.predict(test_fe.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()).collect()

Help on class CustomRegression in module regression.algorithms:

class CustomRegression(MassRegressionAlgorithm)
 |  Custom mass univariate regression algorithm.
 |  
 |  Parameters
 |  ----------
 |  
 |  algorithm : object
 |      An object encapsulating a regression algorithm.
 |  
 |      Can be a scikit-learn algorithm or a custom algorithm that adheres to a
 |      scikit-learn style API. Must implement a fit(X, y) method that takes a
 |      deisgn matrix and response vector (X and y, both NumPy ndarrays) and
 |      returns a fitted model. The fitted model should expose data members
 |      coef_ and intercept_ that contain the fitted coefficients and intercept
 |      (if any). The fitted model should also implement a score(X, y) method
 |      giving the r-squared of fit for the given data and a predict(X) method
 |      that predicts responses given a design matrix.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, algorithm)
 |  
 |  -----------------------------------

In [18]:
alg = MLPRegressor(hidden_layer_sizes=(16,))


In [30]:
from sklearn.datasets import make_regression
X, Y = make_regression(n_samples=100, n_features=3, n_informative=3, n_targets=1, noise=1.0)
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(X, Y.T)

In [20]:
Y = output.select('Target').rdd.map(lambda x: list(x)[0])
X = np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect())
from regression.utils import toseries
y = toseries(Y)

In [ ]:
output.select('Target').rdd.map(lambda x: list(x)[0]).collect()

In [34]:
toseries(Y).map(lambda v: deepcopy(alg).fit(X, v)).first()

array([ MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)], dtype=object)

In [35]:
alg = MLPRegressor(hidden_layer_sizes=(16,))
alg.fit(X, y)

TypeError: Singleton array array(PythonRDD[19] at RDD at PythonRDD.scala:48, dtype=object) cannot be considered a valid collection.

In [29]:
from sklearn.utils.validation import check_X_y

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 16, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-29-42a89fa63232>", line 1, in <lambda>
AttributeError: 'float' object has no attribute 'shape'

	at org.apache.spark.api.python.Python